In [1]:
#!/usr/bin/python
# -*- coding: latin-1 -*-
"""This notebook updates de SCI database from the content of the annexes"""

import os, sys, inspect
from datetime import datetime, timedelta

current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
folder_parts = current_folder.split(os.sep)
pywikibot_folder = os.sep.join(folder_parts[0:-1])

if current_folder not in sys.path:
    sys.path.insert(0, current_folder)
if pywikibot_folder not in sys.path:
    sys.path.insert(0, pywikibot_folder)

import pywikibot as pb
from pywikibot import pagegenerators, textlib
from StringIO import StringIO
import mwparserfromhell as mwh
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
sns.set_style("darkgrid")

%matplotlib inline

In [3]:
BASE_WLE2016_NAME           = u"Commons:Wiki Loves Earth 2016 in Spain"
SCI_DB_PAGE                 = BASE_WLE2016_NAME + u"/SCI DB"

In [4]:
annexes = [
    [u'Anexo:Lugares de importancia comunitaria de Andalucía', u'Andalusia'],
    [u'Anexo:Lugares de importancia comunitaria de Aragón', u'Aragon'],
    [u'Anexo:Lugares de importancia comunitaria de Asturias', u'Asturias'],
    [u'Anexo:Lugares de importancia comunitaria de Cantabria', u'Cantabria'],
    [u'Anexo:Lugares de importancia comunitaria de Castilla-La Mancha', u'Castile-La Mancha'],
    [u'Anexo:Lugares de importancia comunitaria de Castilla y León', u'Castile and León'],
    [u'Anexo:Lugares de importancia comunitaria de Cataluña', u'Catalonia'],
    [u'Anexo:Lugares de importancia comunitaria de la Comunidad de Madrid', u'Community of Madrid'],
    [u'Anexo:Lugares de importancia comunitaria de la Comunidad Valenciana', u'Valencian Community'],
    [u'Anexo:Lugares de importancia comunitaria de Extremadura', u'Extremadura'],
    [u'Anexo:Lugares de importancia comunitaria de las Islas Baleares', u'Balearic Islands'],
    [u'Anexo:Lugares de importancia comunitaria de las Islas Canarias', u'Canary Islands'],
    [u'Anexo:Lugares de importancia comunitaria de Galicia', u'Galicia'],
    [u'Anexo:Lugares de importancia comunitaria de La Rioja', u'La Rioja'],
    [u'Anexo:Lugares de importancia comunitaria de Navarra', u'Navarre'],
    [u'Anexo:Lugares de importancia comunitaria de la Región de Murcia', u'Region of Murcia'],
    [u'Anexo:Lugares de importancia comunitaria del País Vasco', u'Basque Country'],
    [u'Anexo:Lugares de importancia comunitaria de Ceuta y Melilla', u'Ceuta y Melilla'],
    [u'Anexo:Lugares de importancia comunitaria del MAGRAMA', u'MAGRAMA']
]

In [5]:
commons_site = pb.Site("commons", "commons")
wikipedia_site = pb.Site("es", "wikipedia")

sites_df = pd.DataFrame(
    columns=['code', 'commons_cat', 'image'])

for annex in annexes:
    pb.output("Processing annex " + annex[0])
    page = pb.Page(wikipedia_site, annex[0])
    text = page.text
    wikicode = mwh.parse(text)
    templates = [template for template in wikicode.filter_templates() if template.name.lower().strip() == u"fila lic"]
    for template in templates :
        df_row = dict()
        df_row["code"] = template.get(u"código").value.strip()
        df_row["commons_cat"] = u''
        df_row["image"] = u''
        try:
            if template.get(u"categoría-Commons").value :
                df_row["commons_cat"] = template.get(u"categoría-Commons").value.strip()
            if template.get(u"imagen").value :
                df_row["image"] = template.get(u"imagen").value.strip()
        except :
            print template.get(u"código").value.strip()

        sites_df = sites_df.append(df_row, ignore_index=True)

Processing annex Anexo:Lugares de importancia comunitaria de Andalucía
Processing annex Anexo:Lugares de importancia comunitaria de Aragón
Processing annex Anexo:Lugares de importancia comunitaria de Asturias
Processing annex Anexo:Lugares de importancia comunitaria de Cantabria
Processing annex Anexo:Lugares de importancia comunitaria de Castilla-La Mancha
Processing annex Anexo:Lugares de importancia comunitaria de Castilla y León
Processing annex Anexo:Lugares de importancia comunitaria de Cataluña
Processing annex Anexo:Lugares de importancia comunitaria de la Comunidad de Madrid
Processing annex Anexo:Lugares de importancia comunitaria de la Comunidad Valenciana
Processing annex Anexo:Lugares de importancia comunitaria de Extremadura
Processing annex Anexo:Lugares de importancia comunitaria de las Islas Baleares
Processing annex Anexo:Lugares de importancia comunitaria de las Islas Canarias
Processing annex Anexo:Lugares de importancia comunitaria de Galicia
Processing annex Anexo

In [6]:
pb.output('Retrieving --> WLE SCI list')
sci_list_page = pb.Page(commons_site, SCI_DB_PAGE)
sci_list_text = StringIO(sci_list_page.text[sci_list_page.text.find('\n') + 
                                                      1:sci_list_page.text.rfind('\n')])
sci_df = pd.read_csv(sci_list_text, sep=";",
                     index_col=False,
                     names=["name", "code", "magrama_url", "community",
                            "bio_region", "continent", "min_altitude",
                            "max_altitude", "avg_altitude", "longitude",
                            "latitude", "area", "marine_percentage",
                            "marine_area", "image", "commons_cat", "wikidata_id"])
pb.output('Retrieved --> WLE SCI list')

Retrieving --> WLE SCI list
Retrieved --> WLE SCI list


In [7]:
sci_df.drop(['commons_cat', 'image'], axis=1, inplace=True)

In [8]:
columns = ["name", "code", "magrama_url", "community",
                            "bio_region", "continent", "min_altitude",
                            "max_altitude", "avg_altitude", "longitude",
                            "latitude", "area", "marine_percentage",
                            "marine_area", "image", "commons_cat", "wikidata_id"]
result = pd.merge(sci_df, sites_df, on='code', how='outer')
result = result[columns]
csv_text = result.to_csv(sep=';', encoding='utf-8', header=False, index=False)
csv_text = '<pre>\n' + csv_text.replace('.0;', ';') + '\n</pre>'

In [9]:
db_page = pb.Page(commons_site, SCI_DB_PAGE)
db_page.text = unicode(csv_text, "utf-8", "replace")
pb.output('Publishing --> WLE SCI DB update')
db_page.save(u"WLE Spain SCI DB update")

Publishing --> WLE 2017 SCI DB update
Sleeping for 12.3 seconds, 2017-06-16 19:21:07
Page [[Commons:Wiki Loves Earth 2016 in Spain/SCI DB]] saved
